In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import re
from typing import Dict, List, Any
from datetime import datetime

In [3]:
def extractData(bill_text: str) -> Dict[str, Any]:
    return {
        "bill_id": "INS-2025-001234",
        "customer_id": "CUST-5678",
        "total": 500.00,
        "due_date": "2025-12-20",
        "billing_period": "2025-11-01 to 2025-11-30",
        "line_items": [
            {"name": "Base Premium", "amount": 350.00},
            {"name": "Risk Adjustment", "amount": 75.00},
            {"name": "Service Fee", "amount": 25.00},
            {"name": "State Tax", "amount": 50.00}
        ],
        "previous_bill_total": 450.00,
        "policy_type": "Comprehensive Auto Insurance"
    }

In [4]:
def calculate(data: Dict[str, Any]) -> Dict[str, Any]:
    total = data.get("total", 0)
    
    # Calculate payment options
    installment_2 = round(total / 2, 2)
    installment_3 = round(total / 3, 2)
    installment_4 = round(total / 4, 2)
    
    # Calculate potential savings (e.g., discount for full payment)
    full_payment_discount = round(total * 0.02, 2)  # 2% discount
    
    return {
        "total_due": total,
        "payment_options": {
            "full_payment": {
                "amount": total,
                "discount": full_payment_discount,
                "final_amount": total - full_payment_discount,
                "description": "Pay in full and save 2%"
            },
            "two_installments": {
                "amount_per_payment": installment_2,
                "number_of_payments": 2,
                "total": total,
                "description": "Split into 2 equal payments"
            },
            "three_installments": {
                "amount_per_payment": installment_3,
                "number_of_payments": 3,
                "total": total,
                "description": "Split into 3 equal payments"
            },
            "four_installments": {
                "amount_per_payment": installment_4,
                "number_of_payments": 4,
                "total": total,
                "description": "Split into 4 equal payments"
            }
        },
        "auto_pay_discount": round(total * 0.01, 2),  # 1% for auto-pay
        "potential_savings": {
            "increase_deductible": round(total * 0.15, 2),
            "bundle_policies": round(total * 0.10, 2),
            "safe_driver_discount": round(total * 0.05, 2)
        }
    }

In [5]:
def compare(current_bill: Dict[str, Any], previous_bill_total: float) -> Dict[str, Any]:
    current_total = current_bill.get("total", 0)
    difference = current_total - previous_bill_total
    percentage_change = round((difference / previous_bill_total * 100), 2) if previous_bill_total > 0 else 0
    
    # Identify reasons for change
    reasons = []
    if difference > 0:
        if abs(difference) > 50:
            reasons.append("Significant premium adjustment due to risk assessment")
        elif abs(difference) > 20:
            reasons.append("Minor premium adjustment")
        reasons.append("Possible factors: claims history, coverage changes, or rate updates")
    elif difference < 0:
        reasons.append("Premium decreased - you may have qualified for additional discounts")
    else:
        reasons.append("No change from previous billing period")
    
    return {
        "current_total": current_total,
        "previous_total": previous_bill_total,
        "difference": difference,
        "percentage_change": percentage_change,
        "change_direction": "increase" if difference > 0 else "decrease" if difference < 0 else "no_change",
        "reasons": reasons
    }

In [6]:
def askQuestion(question_type: str, context: Dict[str, Any]) -> str:
    questions = {
        "payment_method": "Would you like to pay by credit card, bank account, or check?",
        "installment_preference": "Would you prefer to pay in full or split into installments?",
        "auto_pay": "Would you like to set up automatic payments to save 1%?",
        "coverage_review": "Would you like me to review your coverage for potential savings?",
        "due_date_extension": "Do you need to request an extension on your due date?",
        "dispute": "Would you like to dispute any charges on this bill?"
    }
    return questions.get(question_type, "How can I further assist you?")

In [7]:
def callAPI(action: Dict[str, Any]) -> Dict[str, Any]:
    action_type = action.get("type", "")
    
    print(f"[API CALL] Executing: {action_type}")
    print(f"[API CALL] Parameters: {json.dumps(action.get('parameters', {}), indent=2)}")
    
    # Simulate API responses
    responses = {
        "process_payment": {
            "status": "success",
            "transaction_id": "TXN-" + datetime.now().strftime("%Y%m%d%H%M%S"),
            "message": "Payment processed successfully",
            "confirmation_number": "CONF-987654321"
        },
        "setup_installment": {
            "status": "success",
            "installment_plan_id": "PLAN-" + datetime.now().strftime("%Y%m%d"),
            "message": "Installment plan created successfully"
        },
        "setup_autopay": {
            "status": "success",
            "autopay_id": "AUTO-" + datetime.now().strftime("%Y%m%d"),
            "message": "Auto-pay enrollment successful",
            "discount_applied": True
        },
        "request_extension": {
            "status": "success",
            "new_due_date": "2025-12-30",
            "message": "Due date extension granted"
        }
    }
    
    return responses.get(action_type, {"status": "error", "message": "Unknown action type"})

In [8]:
class InsuranceLLMAssistant:
    def __init__(self, model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"):
        """Initialize the insurance LLM assistant."""
        print("Loading model and tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            load_in_4bit=True,
            torch_dtype="auto"
        )
        print("Model loaded successfully!")
        
        self.conversation_history = []
        self.context_data = {}
        self.system_instruction = self._get_system_instruction()
        
    def _get_system_instruction(self) -> str:
        return """You are an expert insurance billing assistant. Your role is to:

1. EXPLAIN bills clearly by breaking down each line item
2. COMPARE current bills with previous bills and explain changes
3. SUGGEST payment options including installments and discounts
4. RECOMMEND money-saving adjustments (deductibles, bundling, discounts)
5. EXECUTE actions like processing payments or setting up auto-pay when requested

CRITICAL: Always respond in valid JSON format with these keys:
{
    "explanation": "Clear explanation of the bill or situation",
    "bill_breakdown": ["List of line items explained"],
    "comparison": "Comparison with previous bill if applicable",
    "suggested_payment_options": ["List of payment options with amounts"],
    "money_saving_tips": ["Potential savings opportunities"],
    "questions_for_customer": ["Clarifying questions if needed"],
    "action_to_execute": {"type": "action_type", "parameters": {}} or null,
    "next_steps": "What customer should do next"
}

Be friendly, clear, and focused on helping customers understand their bills and save money."""

    def build_prompt(self) -> str:
        prompt = f"[SYSTEM]: {self.system_instruction}\n\n"
        
        # Add context data if available
        if self.context_data:
            prompt += f"[CONTEXT DATA]: {json.dumps(self.context_data, indent=2)}\n\n"
        
        # Add conversation history
        for msg in self.conversation_history:
            role = msg["role"]
            content = msg["content"]
            prompt += f"[{role.upper()}]: {content}\n"
        
        prompt += "[ASSISTANT]: "
        return prompt
    
    def process_user_input(self, user_input: str, bill_data: Dict[str, Any] = None):
        print(f"\n{'='*60}")
        print(f"USER: {user_input}")
        print(f"{'='*60}\n")
        
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_input
        })
        
        # Process bill data if provided
        if bill_data:
            self.context_data["bill_data"] = bill_data
            self.context_data["calculations"] = calculate(bill_data)
            if "previous_bill_total" in bill_data:
                self.context_data["comparison"] = compare(
                    bill_data, 
                    bill_data["previous_bill_total"]
                )
        
        # Build prompt
        prompt_text = self.build_prompt()
        
        # Generate response
        inputs = self.tokenizer(prompt_text, return_tensors="pt").to("cuda")
        attention_mask = inputs.input_ids.ne(self.tokenizer.pad_token_id).long()
        
        print("Generating response...")
        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=attention_mask,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )
        
        raw_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract assistant response
        assistant_response = self._extract_assistant_response(raw_response)
        
        # Parse and execute
        structured_response = self._parse_and_execute(assistant_response)
        
        # Add to history
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_response
        })
        
        return structured_response
    
    def _extract_assistant_response(self, raw_response: str) -> str:
        # Find the last [ASSISTANT]: marker
        parts = raw_response.split("[ASSISTANT]:")
        if len(parts) > 1:
            return parts[-1].strip()
        return raw_response.strip()
    
    def _parse_and_execute(self, response: str) -> Dict[str, Any]:
        try:
            # Try to extract JSON from response
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                response_json = json.loads(json_match.group())
            else:
                # If no JSON found, create a basic structure
                response_json = {
                    "explanation": response,
                    "action_to_execute": None
                }
            
            # Display response
            self._display_response(response_json)
            
            # Execute action if specified
            if response_json.get("action_to_execute"):
                action_result = callAPI(response_json["action_to_execute"])
                print(f"\n[ACTION RESULT]: {json.dumps(action_result, indent=2)}")
                response_json["action_result"] = action_result
            
            return response_json
            
        except Exception as e:
            print(f"[ERROR] Failed to parse response: {e}")
            print(f"[RAW RESPONSE]: {response}")
            return {
                "explanation": response,
                "error": str(e)
            }
    
    def _display_response(self, response_json: Dict[str, Any]):
        
        print("\n" + "="*60)
        print("ASSISTANT RESPONSE")
        print("="*60)
        
        if "explanation" in response_json:
            print(f"\n📋 EXPLANATION:\n{response_json['explanation']}")
        
        if "bill_breakdown" in response_json:
            print(f"\n💰 BILL BREAKDOWN:")
            for item in response_json["bill_breakdown"]:
                print(f"  • {item}")
        
        if "comparison" in response_json:
            print(f"\n📊 COMPARISON:\n{response_json['comparison']}")
        
        if "suggested_payment_options" in response_json:
            print(f"\n💳 PAYMENT OPTIONS:")
            for option in response_json["suggested_payment_options"]:
                print(f"  • {option}")
        
        if "money_saving_tips" in response_json:
            print(f"\n💡 MONEY-SAVING TIPS:")
            for tip in response_json["money_saving_tips"]:
                print(f"  • {tip}")
        
        if "questions_for_customer" in response_json:
            print(f"\n❓ QUESTIONS FOR YOU:")
            for question in response_json["questions_for_customer"]:
                print(f"  • {question}")
        
        if "next_steps" in response_json:
            print(f"\n➡️  NEXT STEPS:\n{response_json['next_steps']}")
        
        print("\n" + "="*60 + "\n")

In [9]:
if __name__ == "__main__":
    # Initialize assistant
    assistant = InsuranceLLMAssistant()
    
    # Example 1: Customer wants to understand their bill
    print("\n" + "="*60)
    print("SCENARIO 1: Understanding the Bill")
    print("="*60)
    
    bill_data = extractData("customer bill text here")
    assistant.process_user_input(
        "I received my insurance bill and want to understand the charges.",
        bill_data=bill_data
    )
    
    # Example 2: Customer asks about payment options
    print("\n" + "="*60)
    print("SCENARIO 2: Payment Options")
    print("="*60)
    
    assistant.process_user_input(
        "Can I split this payment into installments? I'd like to see my options."
    )
    
    # Example 3: Customer wants to pay
    print("\n" + "="*60)
    print("SCENARIO 3: Processing Payment")
    print("="*60)
    
    assistant.process_user_input(
        "I'd like to pay in full with my credit card and set up auto-pay for future bills."
    )
    
    # Example 4: Customer asks about savings
    print("\n" + "="*60)
    print("SCENARIO 4: Money-Saving Tips")
    print("="*60)
    
    assistant.process_user_input(
        "My bill went up by $50. How can I reduce my insurance costs?"
    )

Loading model and tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Model loaded successfully!

SCENARIO 1: Understanding the Bill

USER: I received my insurance bill and want to understand the charges.

Generating response...

ASSISTANT RESPONSE

📋 EXPLANATION:
2025-11-01 through 2025-11-30 is your billing period, so your total due is $500.0 for this month. The following table will break down that amount into a couple different line items.

|Line Item||Amount|
|---|---|
|Base Premium||$350.0|
|Risk Adjustment||$75.0|
|Service Fee||$25.0|
|State Tax||$50.0|

The previous bill's total was $450.0; there's an increase of $50.0 in this month's bill.

For comparison, let’s see what our previous bill was. It shows you have made one change from last year—your base premium has increased by $25.0.

You can pay in full now, but there are other ways to split the cost out over multiple installments. There are also other discounts available. We’ll provide some additional information about those later. Do you want to make sure it all adds up before we finalize? Let 